In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from IPython.display import IFrame

view-source:https://covid19.sokat.ai/

In [2]:
IFrame("https://covid19.sokat.ai/", width="100%", height=400)

In [3]:
def markers_to_dataframe(url):
    """This function takes the https://covid19.sokat.ai/ web map as input to generate a CSV file of drive-thru testing locations. 
    """
    # Download webpage data with requests
    page = requests.get(url)
    # Rarse the webpage data with BeautifulSoup
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Select scripts in HTML file
    scripts = list(soup.find_all('script'))
    
    # Subset for the script with marker data and begin string splitting
    html = str(list(scripts[7])).split('var marker')
    
    # Use for-loops to generate the names, coordinates, and URLs of drive-thru testing locations
    list_of_names = []
    for i in range(2,142):
        list_of_names.append(html[i].split('target="_blank">')[1].split('</a></div>')[0])
    
    list_of_coords = []
    for i in range(2,142):
        list_of_coords.append(html[i].split('target="_blank">')[0].split("L.marker(\\n                [")[1].split("]")[0])

    list_of_urls = []
    for i in range(2,142):
        list_of_urls.append(html[i].split('target="_blank">')[0].split('><a href="')[1].split('"')[0])

    # Concatenate lists into a pandas DataFrame
    dataframe = pd.DataFrame([list_of_names, list_of_coords, list_of_urls]).T
    
    # Rename the columns with appropriate titles
    dataframe.columns = ["Name","Coordinate", "URL"]
    
    # Generate Latitude and Longitude Series with string splitting
    dataframe["Latitude"] = dataframe["Coordinate"].str.split(expand=True)[0].str.replace(",","").astype(float)
    dataframe["Longitude"] = dataframe["Coordinate"].str.split(expand=True)[1].astype(float)
    
    # Return results
    return dataframe

In [4]:
df = markers_to_dataframe("https://covid19.sokat.ai/")
df

,Name,Coordinate,URL,Latitude,Longitude
0,collaborative effort between health care prov...,"61.1828699, -149.837269",https://www.adn.com/alaska-news/anchorage/2020...,61.182870,-149.837269
1,Church of the Highlands- Grant\'s Mill,"33.51699910000001, -86.6558468",https://www.google.com/amp/s/www.wvtm13.com/am...,33.516999,-86.655847
2,UAMS,"34.749005100000005, -92.32009649999999",https://www.thv11.com/mobile/article/news/heal...,34.749005,-92.320096
3,Mayo Clinic Hospital,"33.66211083, -111.95351289999999",https://www.12news.com/article/news/health/cor...,33.662111,-111.953513
4,Mayo Clinic,"33.586042299999995, -111.7940787",https://www.12news.com/article/news/health/cor...,33.586042,-111.794079
...,...,...,...,...,...
135,University of Chicago Medical Center,"41.7882, -87.60449",https://www.nbcchicago.com/news/local/drive-th...,41.788200,-87.604490
136,Advocate Lutheran General Hospital,"42.039371, -87.847572",https://www.nbcchicago.com/news/local/drive-th...,42.039371,-87.847572
137,Deaconess Midtown Hospital,"37.98316, -87.57121",https://www.courierpress.com/story/news/2020/0...,37.983160,-87.571210
138,North Iowa Events Center,"43.155429999999996, -93.20428000000001",https://kttc.com/2020/03/18/drive-through-test...,43.155430,-93.204280


In [5]:
df.to_csv("us-drive-thru-testing-locations.csv")